# NOTE

For Spark 3 Cosmos DB connector has slightly different configuration. Please select different snippest if a Spark 3.1 + pool is attached.

In [2]:
# Load a streaming Spark DataFrame from a Cosmos DB container
# To select a preferred list of regions in a multi-region Cosmos DB account, add .option("spark.cosmos.preferredRegions", "<Region1>,<Region2>")

# For Spark 2.4
# dfStream = spark.readStream\
#     .format("cosmos.oltp")\
#     .option("spark.synapse.linkedService", "CosmosDbNoSql1")\
#     .option("spark.cosmos.container", "orders")\
#     .option("spark.cosmos.changeFeed.readEnabled", "true")\
#     .option("spark.cosmos.changeFeed.startFromTheBeginning", "true")\
#     .option("spark.cosmos.changeFeed.checkpointLocation", "/localReadCheckpointFolder")\
#     .option("spark.cosmos.changeFeed.queryName", "streamQuery")\
#     .load()

# For Spark 3.1 +
dfStream = spark.readStream\
    .format("cosmos.oltp.changeFeed")\
    .option("spark.synapse.linkedService", "CosmosDbNoSql1")\
    .option("spark.cosmos.container", "orders")\
    .option("spark.cosmos.changeFeed.startFrom", "Beginning")\
    .option("spark.cosmos.changeFeed.mode", "Incremental")\
    .load()

In [3]:
from   pyspark.sql.functions import * 
from pyspark.sql import *
from delta.tables import *

In [4]:
dfStream

In [5]:
dfStream=dfStream.withColumn("Order_Date",to_date("Order_Timestamp"))

In [6]:
dfStream=dfStream.withColumn("Order_Timestamp",to_timestamp("Order_Timestamp"))

In [7]:
dfStream

In [8]:
dfStream1=dfStream.withWatermark('Order_Timestamp', '10 minutes').groupBy("Item_Id","Order_Date",window("Order_Timestamp","6 minutes")).agg(sum("qty")).alias("sum_qty")

In [9]:
dfStream1=dfStream1.withColumnRenamed("sum(qty)","sum_qty")

In [10]:
dfStream1=dfStream1.withColumn("window_Start_Time",expr("window.start"))
dfStream1=dfStream1.withColumn("window_end_Time",expr("window.end"))

In [11]:
def microbatch(batch_df,batch_id):
    delta_table=DeltaTable.forPath(spark, 'abfss://streamsynapse@synapasestream.dfs.core.windows.net/synapse/workspaces/streamoutput/tumbling_window')
    #print(delta_table)
    delta_table.alias('target').merge(batch_df.alias('updates'),'target.item_id = updates.item_id and target.window_Start_Time=updates.window_Start_Time and target.window_end_Time=updates.window_end_Time and target.order_date=updates.order_date').whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

In [12]:
#dfStream1.writeStream.outputMode("complete").format("delta").option("path","abfss://streamsynapse@synapasestream.dfs.core.windows.net/synapse/workspaces/streamoutput/tumbling_window").option("checkpointLocation","abfss://streamsynapse@synapasestream.dfs.core.windows.net/synapse/workspaces/tumb_checkpoint/").start()

In [13]:
dfStream1.writeStream.outputMode("complete").foreachBatch(microbatch).option("checkpointLocation","abfss://streamsynapse@synapasestream.dfs.core.windows.net/synapse/workspaces/tumb_checkpoint/").start()

In [16]:
%%sql

select item_id,order_date,WINDOW_start_time,window_end_time,sum_qty from delta.`abfss://streamsynapse@synapasestream.dfs.core.windows.net/synapse/workspaces/streamoutput/tumbling_window` order by WINDOW_start_time desc;